<a href="https://colab.research.google.com/github/ezeguins/Bounding-Box-Detecction-VGG16/blob/main/predict_txt_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BOUNDING BOX MODEL PREDICTION FROM .TXT FILE CONTAINING TEST IMAGES FILENAMES GENERATED IN THE "MODEL_TRAINING.TS" FILE

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
version = 19
detector = 'detector' + str(version) + '.h5'
plot = 'plot' + str(version) + '.png'
test_images = 'test_images' + str(version) + '.txt'

BASE_PATH = "/content/gdrive/MyDrive/VGG16/dataset"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "etiquetas_export-data-60_solobloques.csv"])

# define the path to the base output directory
BASE_OUTPUT = "/content/gdrive/MyDrive/VGG16/output"
# define the path to the output serialized model, model training plot,
# and testing image filenames
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, detector])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, plot])
TEST_FILENAMES = os.path.sep.join([BASE_OUTPUT, test_images])

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import mimetypes

import imutils
import cv2
import os

In [ ]:
outfile = "/content/gdrive/MyDrive/VGG16/dataset/datos_full.npz"
npzfile = np.load(outfile)
data1 = npzfile['arr_0']
targets1 = npzfile['arr_1']
filenames1 = npzfile['arr_2']

In [ ]:
args = {"input": TEST_FILENAMES}
filetype = mimetypes.guess_type(args["input"])[0]
imagePaths = [args["input"]]
imagePaths

['/content/gdrive/MyDrive/VGG16/output/test_images19.txt']

In [ ]:
# determine the input file type, but assume that we're working with
# single input image
filenames = open(args["input"]).read().strip().split("\n")
fn_index = list(range(0,len(filenames),1))
print(type(fn_index))

# if the file type is a text file, then we need to process *multiple*
# images
if "text/plain" == filetype:
	# load the filenames in our testing file and initialize our list
	# of image paths
	filenames = open(args["input"]).read().strip().split("\n")

	# loop over the filenames
	for index, file in enumerate(filenames):
		for idx, f in enumerate(filenames1):
			if file == f:
				fn_index[index] = idx 
print(len(fn_index))




<class 'list'>
172


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# load our trained bounding box regressor from disk
print("[INFO] loading object detector...")
model = load_model(MODEL_PATH)
# loop over the images that we'll be testing using our bounding box
# regression model
for index in fn_index:
  # load the input image (in Keras format) from disk and preprocess
  # it, scaling the pixel intensities to the range [0, 1]
  image = data1[index]
  image2 = np.expand_dims(image, axis=0)
  print(image2.shape)
  preds = model.predict(image2)[0]
  (startX, startY, endX, endY) = preds
  image = image * 255
  w, h = 224, 224
  startX = int(startX * w)
  startY = int(startY * h)
  endX = int(endX * w)
  endY = int(endY * h)
  # draw the predicted bounding box on the image
  cv2.rectangle(image, (startX, startY), (endX, endY),	(0, 255, 0), 2)
  # show the output image
  cv2_imshow(image)
  print(filenames1[index])
  cv2.waitKey(0)